In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('savant_data.csv')

In [3]:
df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,FF,2023-08-08,96.2,2.65,5.09,"Arias, Gabriel",672356,579328,NaN,ball,...,0,0,1,1,0,Standard,Standard,134,0.0,0.023
1,FF,2023-08-08,95.5,2.61,5.07,"Arias, Gabriel",672356,579328,NaN,called_strike,...,0,0,1,1,0,Standard,Standard,131,0.0,-0.066
2,CU,2023-08-08,82.6,1.92,5.50,"Arias, Gabriel",672356,579328,NaN,called_strike,...,0,0,1,1,0,Standard,Standard,301,0.0,-0.058
3,CU,2023-08-08,80.6,2.01,5.48,"Rocchio, Brayan",677587,579328,NaN,called_strike,...,0,0,1,1,0,Standard,Standard,310,0.0,-0.033
4,FF,2023-08-08,95.3,2.59,5.07,"Rocchio, Brayan",677587,579328,NaN,ball,...,0,0,1,1,0,Standard,Standard,132,0.0,0.022


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 92 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   pitch_type                       415 non-null    object 
 1   game_date                        415 non-null    object 
 2   release_speed                    415 non-null    float64
 3   release_pos_x                    415 non-null    float64
 4   release_pos_z                    415 non-null    float64
 5   player_name                      415 non-null    object 
 6   batter                           415 non-null    int64  
 7   pitcher                          415 non-null    int64  
 8   events                           22 non-null     object 
 9   description                      415 non-null    object 
 10  spin_dir                         0 non-null      float64
 11  spin_rate_deprecated             0 non-null      float64
 12  break_angle_deprecated

In [9]:
df_missed = df[['player_name','description','zone','balls','strikes']]

In [10]:
df_missed.head()

,player_name,description,zone,balls,strikes
0,"Arias, Gabriel",ball,12,0,2
1,"Arias, Gabriel",called_strike,8,0,1
2,"Arias, Gabriel",called_strike,8,0,0
3,"Rocchio, Brayan",called_strike,9,2,1
4,"Rocchio, Brayan",ball,13,1,1


In [21]:
def actual_strike(df):
    if df['zone'] > 10:
        return 'ball'
    else:
        return 'called_strike'

In [22]:
df_missed['actual_description'] = df_missed.apply(actual_strike,axis=1)

<ipython-input-22-6039ef51c0b9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_missed['actual_description'] = df_missed.apply(actual_strike,axis=1)


In [23]:
df_missed.head()

,player_name,description,zone,balls,strikes,actual_description
0,"Arias, Gabriel",ball,12,0,2,ball
1,"Arias, Gabriel",called_strike,8,0,1,called_strike
2,"Arias, Gabriel",called_strike,8,0,0,called_strike
3,"Rocchio, Brayan",called_strike,9,2,1,called_strike
4,"Rocchio, Brayan",ball,13,1,1,ball


In [25]:
def incorrect_call(df):
    if (df['description'] != df['actual_description']) and (df['actual_description'] == 'ball'):
        return 'missed ball'
    elif (df['description'] != df['actual_description']) and (df['actual_description'] == 'called_strike'):
        return 'missed strike'
    else:
        return 'correct'

In [26]:
df_missed['incorrect_call'] = df_missed.apply(incorrect_call,axis=1)

<ipython-input-26-b264065a4cbd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_missed['incorrect_call'] = df_missed.apply(incorrect_call,axis=1)


In [27]:
df_missed.head()

,player_name,description,zone,balls,strikes,actual_description,incorrect_call
0,"Arias, Gabriel",ball,12,0,2,ball,correct
1,"Arias, Gabriel",called_strike,8,0,1,called_strike,correct
2,"Arias, Gabriel",called_strike,8,0,0,called_strike,correct
3,"Rocchio, Brayan",called_strike,9,2,1,called_strike,correct
4,"Rocchio, Brayan",ball,13,1,1,ball,correct


In [29]:
df_missed['incorrect_call'].value_counts()

correct          383
missed strike     17
missed ball       15
Name: incorrect_call, dtype: int64

In [39]:
df_missed = df_missed[df_missed['incorrect_call'] != 'correct']

In [40]:
df_missed.groupby(by=['player_name'])['incorrect_call'].value_counts()

player_name      incorrect_call
Arias, Gabriel   missed ball       1
                 missed strike     1
Brennan, Will    missed strike     1
Calhoun, Kole    missed ball       1
Giménez, Andrés  missed strike     3
                 missed ball       1
Gonzalez, Oscar  missed ball       2
                 missed strike     2
Kwan, Steven     missed ball       5
                 missed strike     3
Laureano, Ramón  missed ball       1
Naylor, Bo       missed strike     2
Ramírez, José    missed strike     1
Rocchio, Brayan  missed ball       2
                 missed strike     2
Straw, Myles     missed ball       2
                 missed strike     1
Tena, José       missed strike     1
Name: incorrect_call, dtype: int64

In [45]:
def post_pitch_balls(df):
    if df['description'] == 'ball':
        return df['balls']+1
    else:
        return df['balls']

In [46]:
def post_pitch_strikes(df):
    if df['description'] == 'called_strike':
        return df['strikes']+1
    else:
        return df['strikes']

In [48]:
df_missed['post_pitch_balls'] = df_missed.apply(post_pitch_balls,axis=1)
df_missed['post_pitch_strikes'] = df_missed.apply(post_pitch_strikes,axis=1)

<ipython-input-48-b98d05ee8b3f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_missed['post_pitch_balls'] = df_missed.apply(post_pitch_balls,axis=1)
<ipython-input-48-b98d05ee8b3f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_missed['post_pitch_strikes'] = df_missed.apply(post_pitch_strikes,axis=1)


In [50]:
def hypo_balls(df):
    if df['incorrect_call'] == 'missed ball':
        return df['balls']+1
    else:
        return df['balls']

In [51]:
def hypo_strikes(df):
    if df['incorrect_call'] == 'missed strike':
        return df['strikes']+1
    else:
        return df['strikes']

In [52]:
df_missed['hypo_balls'] = df_missed.apply(hypo_balls,axis=1)
df_missed['hypo_strikes'] = df_missed.apply(hypo_strikes,axis=1)

<ipython-input-52-28b92090a202>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_missed['hypo_balls'] = df_missed.apply(hypo_balls,axis=1)
<ipython-input-52-28b92090a202>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_missed['hypo_strikes'] = df_missed.apply(hypo_strikes,axis=1)


In [53]:
df_missed.head()

,player_name,description,zone,balls,strikes,actual_description,incorrect_call,post_pitch_balls,post_pitch_strikes,hypo_balls,hypo_strikes
6,"Laureano, Ramón",called_strike,11,2,2,ball,missed ball,2,3,3,2
10,"Gonzalez, Oscar",called_strike,13,1,1,ball,missed ball,1,2,2,1
26,"Gonzalez, Oscar",ball,9,1,1,called_strike,missed strike,2,1,1,2
46,"Kwan, Steven",ball,7,0,0,called_strike,missed strike,1,0,0,1
57,"Giménez, Andrés",ball,8,2,2,called_strike,missed strike,3,2,2,3
